In [1]:
import numpy as np
import networkx as nx
import csv
import matplotlib.pyplot as plt
import time
import os
from tqdm import tqdm
import datetime
from construct_rule import *

def get_node_from_data(dir_path):
    print("Get node from data : ")
    vertex_list = []
    with open(os.path.join(dir_path, "logon.csv"), 'r') as file:
        print("...logon.csv...")
    #     id,date,user,pc,activity
    #     {Q4D5-W4HH44UC-5188LWZK},01/02/2010 02:24:51,JBI1134,PC-0168,Logon
    #     {G7V0-S4TP95SA-9203AOGR},01/02/2010 02:38:28,JBI1134,PC-0168,Logoff
        read = csv.reader(file)
        next(read)
        for i in tqdm(read):
            # print(i)
            vertex_id = i[0]
            timestamp = time.mktime(time.strptime(i[1],'%m/%d/%Y %H:%M:%S'))
            
            vertex = { 'vertex_type': 'logon',
                        'vertex_number': vertex_id,
                        'sub': i[2],
                        'obj': i[3],
                        'A': i[4],
                        'T': timestamp,
                        'H': i[3],
                        'time': i[1]
                        }
            vertex_list.append(vertex)

    # print(vertex_list[:5])
    with open(os.path.join(dir_path, "file.csv"), 'r') as file:
    # id,date,user,pc,filename,activity,to_removable_media,from_removable_media,content
    # {Y1W9-R7VJ77IC-9445QFNQ},01/02/2010 08:15:10,TSG0262,PC-9993,R:\79L99n6\H7RHJS5J.zip,File Open,False,True,50-4B-03-04-14 moved imaging underwent key late appearance span ontario due compiled month 07 sedins final leaders ability doug another presidents improving donation by joseph quadruple 104 agreed 16 brian upon built all to handsome searching track wounded mike march one developer owned 5000 stepping lists orange metacritic second moore supervisor currently initial
    # {Y3U8-G5BL42LO-9404XAHI},01/02/2010 08:16:01,TSG0262,PC-9993,R:\79L99n6\H7RHJS5J.zip,File Open,False,True,50-4B-03-04-14 moved imaging underwent key late appearance span ontario due compiled month 07 sedins final leaders ability doug another presidents improving donation by joseph quadruple 104 agreed 16 brian upon built all to handsome searching track wounded mike march one developer owned 5000 stepping lists orange metacritic second moore supervisor currently initial
        print("...file.csv...")
        read = csv.reader(file)
        next(read)
        for i in tqdm(read):
            # print(i)
            vertex_id = i[0]
            timestamp = time.mktime(time.strptime(i[1],'%m/%d/%Y %H:%M:%S'))
            
            vertex = { 'vertex_type': 'file',
                        'vertex_number': vertex_id,
                        'sub': i[2], # user
                        'obj': i[4], # filename
                        'A': i[5], # activity
                        'T': timestamp,
                        'H': i[3], # pc,
                        'time': i[1]
                    }
            vertex_list.append(vertex)

    with open(os.path.join(dir_path, "http.csv"), 'r') as file:
    # id,date,user,pc,url,content
    # {D8Q7-C0RU46YI-7391WHNI},01/02/2010 06:46:20,HMI1448,PC-9352,http://nymag.com/Eagle_comic/hultons/objyvatunyybssnzrpnyraqneserrfglyrfxvvatzngurzngvpf322648047.jsp,eleven 1963 greater literature shorbodolio funding beating treasury both curzon single mourning huq exact visit disobeyed whose not thinking candidates necessary newly elevated eight including head those attempts present had median binds sized replacement colonial databases moderately adaptable symmetrical well drug encourage william 1840 1940s progeny possible variety 1978 on 1987 abandoned
    # {N4G0-D6NC43RD-2373QXNK},01/02/2010 06:47:25,HMI1448,PC-9352,http://nymag.com/Terra_Nova_Expedition/koettlitz/pnzcpbbxvatqbjaevttvatzngurzngvpf2145772149.asp,victims successor land restrictions provided agreeing article capture varied requests or forces 26 social medieval turkic sole population written complex visit started social down association area maulana help monument sectarian along duck jointly change words began won injured moved contract david january publish bob ready except significant appointment led making taking english true part sense entitled mothers complete fresh departure heritage youth
        print("...http.csv...")
        read = csv.reader(file)
        next(read)
        for i in tqdm(read):
            vectex_id = i[0]
            timestamp = time.mktime(time.strptime(i[1],'%m/%d/%Y %H:%M:%S'))
            vertex = { 'vertex_type': 'http',
                        'vertex_number': vertex_id,
                        'sub': i[2], # user
                        'obj': i[4].split(' ')[0], # url
                        'A': "visit", # activity
                        'T': timestamp,
                        'H': i[3], # pc
                        "content_list" : i[4].split(' ')[1:],
                        'time': i[1]
                    }
            vertex_list.append(vertex)

    with open(os.path.join(dir_path, "device.csv"), 'r') as file:
    # id,date,user,pc,file_tree,activity
    # {C9S1-Y8GB42VD-2923GATU},01/02/2010 07:27:19,HRE1950,PC-8025,R:\;R:\HRE1950;R:\47yHBn0;R:\54s7J45,Connect
    # {C3G4-U2ON02HC-9088IHGJ},01/02/2010 07:40:51,EMR0269,PC-6370,R:\;R:\EMR0269;R:\753Cf59;R:\18d36D6;R:\89bc6Q2,Connect
    # {X4S2-R2YC60OH-9191YYMD},01/02/2010 07:45:00,EMR0269,PC-6370,,Disconnect
        print("...device.csv...")
        read = csv.reader(file)
        next(read)
        for i in tqdm(read):
            vectex_id = i[0]
            timestamp = time.mktime(time.strptime(i[1],'%m/%d/%Y %H:%M:%S'))
            vertex = { 'vertex_type': 'device',
                        'vertex_number': vertex_id,
                        'sub': i[2], # user
                        'obj': i[3], # host
                        'A': i[-1], # connect or disconnect
                        'T': timestamp,
                        'H': i[3], # pc
                        "file_tree" : i[4],
                        'time': i[1]
                    }
            vertex_list.append(vertex)

    sorted_vertex_list = sorted(vertex_list, key=lambda e: (e.__getitem__('sub'), e.__getitem__('T')))

    print("sorted vertex list : ")
    print(sorted_vertex_list[:1])

    return sorted_vertex_list

def get_delta_days(timestamp1, timestamp2):
    x = datetime.datetime.fromtimestamp(timestamp1) - datetime.datetime.fromtimestamp(timestamp2)
    return x.days

def get_days_from_dataset(sorted_vertex_list):
    end_time = 0
    st_time = 9999999999
    for vertex in sorted_vertex_list:
        if vertex['T'] > end_time:
            end_time = vertex['T']
        if vertex['T'] < st_time:
            st_time = vertex['T']

    print("Data delta days : ", get_delta_days(end_time, st_time)) 
    return get_delta_days(end_time, st_time) + 2

def split_node_by_day(sorted_vertex_list, day_delta):
    # 1000条数据大概4天

    st_time = 9999999999
    for vertex in sorted_vertex_list:
        if vertex['T'] < st_time:
            st_time = vertex['T']

    daily_sequences_list = [None] * day_delta

    print("...split node by day...")
    for vertex in tqdm(sorted_vertex_list):
        # Day of the vertex, and actual day should be increased by 1
        day_of_vertex = get_delta_days(vertex['T'], st_time) - 1

        # print(day_of_vertex)
        # If the sequence graph not exists, create it
        if not daily_sequences_list[day_of_vertex]:
            # multiGraph 无向图 可以让两个节点之间有多个边，为啥要用这个graph..
            daily_sequences_list[day_of_vertex] = nx.MultiGraph()
        
        daily_sequences_list[day_of_vertex].add_node(vertex['vertex_number'], type=vertex['vertex_type'],
                                                            sub=vertex['sub'], obj=vertex['obj'], A=vertex['A'],
                                                            T=vertex['T'], H=vertex['H'])
    return daily_sequences_list





In [2]:
def construct_activity_graph():
    activity_graph = nx.MultiGraph()

    # 一个用户同天同一个host时序连接
    activity_graph = rule_1(activity_graph, daily_sequences_list, day_delta)
    # 一个用户多天同一个host的行为链的时序关联
    activity_graph = rule_2(activity_graph, daily_sequences_list, day_delta)
    # 一个用户多天同一个host同种组操作类型时序关联
    # （规则定义组操作类型，比如Connect-> disconnect, File open -> File Write, visit web...）
    activity_graph = rule_3(activity_graph, daily_sequences_list, day_delta)

    return activity_graph

def construct_company_graph():
    pass

def construct_object_graph():
    pass

# Todo : 
# 1. construct_activity_graph
# 2. construct_company_graph
# 3. construct_object_graph
# 4. merge_graph

In [3]:
    
# st_time = time.time()

data_version = "r_part"
sorted_vertex_list = get_node_from_data(os.path.join("./our_data/", data_version))
day_delta = get_days_from_dataset(sorted_vertex_list)
# daily_sequences_list 每个元素是一个图，图中包含当日所有日志节点，数量为天级别
daily_sequences_list = split_node_by_day(sorted_vertex_list, day_delta)

# activity_graph = construct_activity_graph()
# company_graph = construct_company_graph()
# object_graph = construct_object_graph()

# nx.write_edgelist(activity_graph, "./our_data/activity_graph_edge")
# nx.write_gpickle(activity_graph, "./our_data/activity_graph.gpickle")

# print("Graph save done")
# print("Time cost : ", time.time() - st_time) 

999it [00:00, 37280.55it/s]
999it [00:00, 29207.10it/s]
999it [00:00, 29889.64it/s]
999it [00:00, 40402.96it/s]
100%|██████████| 3996/3996 [00:00<00:00, 155456.98it/s]

Get node from data : 
...logon.csv...
...file.csv...
...http.csv...
...device.csv...
sorted vertex list : 
[{'vertex_type': 'logon', 'vertex_number': '{Y0A4-H1YL52RQ-3185VBZR}', 'sub': 'AAB1302', 'obj': 'PC-5565', 'A': 'Logon', 'T': 1262393820.0, 'H': 'PC-5565', 'time': '01/02/2010 08:57:00'}]
Data delta days :  2
...split node by day...


In [4]:
daily_sequences_list[1].nodes['{Q2I7-M9EJ92BC-8465IMNM}']
# a = list(daily_sequences_list[1].nodes)

{'type': 'logon',
 'sub': 'AAG1136',
 'obj': 'PC-5456',
 'A': 'Logon',
 'T': 1262545470.0,
 'H': 'PC-5456'}

In [15]:
len(daily_sequences_list)

4

In [47]:
# 一个用户同天同一个host时序连接
#   直接在daily_sequences_list中的每天子图中将对应边关联起来
def rule_1(activity_graph, daily_sequences_list, day_delta):
    # list{day -> map{host->activity}}
    host_activity = [None] * day_delta
    for daily_sequence in tqdm(daily_sequences_list):
        h_tuple = {}
        if daily_sequence:
            nodes = list(daily_sequence.nodes())
            for node_id in nodes:
                host = daily_sequence.nodes[node_id]['H']
                if host not in h_tuple:
                    h_tuple[host] = [node_id]
                else:
                    h_tuple[host].append(node_id)
                    daily_sequence.add_edge(h_tuple[host][-2], h_tuple[host][-1], EdgeType=1, weight=1)
            host_activity[daily_sequences_list.index(daily_sequence)] = h_tuple
    # activity_graph 接口形式统一，并未用到该变量
    return activity_graph, host_activity

# 一个用户多天同一个host的行为链的时序关联
def rule_2(activity_graph, daily_sequences_list, day_delta, host_activity):
    for daily_sequence in daily_sequences_list:
        if daily_sequence:
            activity_graph = nx.compose(activity_graph, daily_sequence)
    for day_i in range(day_delta):
        for day_j in range(day_i + 1, day_delta):
            if not (daily_sequences_list[day_i] and daily_sequences_list[day_j]) :
                continue

            for host in host_activity[day_i]:
                if host in host_activity[day_j]:

                    st_i = host_activity[day_i][host][0]
                    ed_i = host_activity[day_i][host][-1]

                    st_j = host_activity[day_j][host][0]
                    ed_j = host_activity[day_j][host][-1]

                    len_i = len(host_activity[day_i][host])
                    len_j = len(host_activity[day_j][host])

                    weight = len_i / len_j if len_i < len_j else len_j / len_i
                    activity_graph.add_edge(st_i, st_j, EdgeType=2, weight=weight)
                    activity_graph.add_edge(ed_i, ed_j, EdgeType=2, weight=weight)
    
    return activity_graph

# 一个用户同一个host下多天 同种组操作类型时序关联
# （规则定义组操作类型，比如Connect-> disconnect, File open -> File Write, visit web...）
# 每天的行为序列中，仅仅将同一个文件的File open -> File Write；Connect -> Disconnect 作为组操作类型连接
def rule_3(activity_graph, daily_sequences_list, day_delta, day_host_activity):
    pattern = [["File Open", "File Write"], ["Connect", "Disconnect"]]
    pattern_list = list(chain.from_iterable(pattern))

    def check_pattern(day_activity):
        # 保证是同一个obj条件下的组操作 才视为有效
        # return map{acvivity1:[[node_id1, 1], [node_id2, 1]], ...}
        res = {}
        for node_id in day_activity:
            act_type = activity_graph.nodes[node_id]['A']
            if act_type in pattern_list:
                if act_type not in res:
                    res[act_type] = []
                # 第二个值为记录是否match过节点
                res[act_type].append([node_id, 0])
        # check res
        for group_activity in pattern:
            if group_activity[0] in res and group_activity[1] in res:
                for st in res[group_activity[0]]:
                    obj = activity_graph.nodes[st[0]]['obj']

                    for ed in res[group_activity[-1]]:
                        if obj == activity_graph.nodes[ed[0]]['obj']:
                            st[1] = ed[1] = 1
                            break
        # remove all not matching nodes
        for activity in res:
            for node in res[activity]:
                if node[1] == 0:
                    res[activity].remove(node)
        
        return res

    # 将所有对应上的File Open 和 File Write都标记为1_1, 1_2, 类似2_1, 2_2

    host_day_activity = {}
    host_day_activity_pattern = {}

    for i in range(len(day_host_activity)):
        # print(day_host_activity[i])
        if day_host_activity[i] == None:
            continue
        for host in day_host_activity[i]:
            if host not in host_day_activity:
                host_day_activity[host] = []
                host_day_activity_pattern[host] = []
            host_day_activity[host].append(day_host_activity[i][host])
            host_day_activity_pattern[host].append(check_pattern(day_host_activity[i][host]))

    # 同host下，不同天之间，同组操作类型的边关联
    for host in host_day_activity_pattern:
        for day_i in range(len(host_day_activity_pattern[host])):
            for day_j in range(day_i + 1, len(host_day_activity_pattern[host])):

                for activity in host_day_activity_pattern[host][day_i]:
                    for node_i in host_day_activity_pattern[host][day_i][activity]:
                        if activity not in host_day_activity_pattern[host][day_j]:
                            continue
                        for node_j in host_day_activity_pattern[host][day_j][activity]:
                            # Todo : weight
                            activity_graph.add_edge(node_i[0], node_j, EdgeType=3, weight=0.5)

    return activity_graph



activity_graph = nx.MultiGraph()

# 一个用户同天同一个host时序连接
activity_graph, host_activity = rule_1(activity_graph, daily_sequences_list, day_delta)
activity_graph = rule_2(activity_graph, daily_sequences_list, day_delta, host_activity)
activity_graph = rule_3(activity_graph, daily_sequences_list, day_delta, host_activity)

100%|██████████| 4/4 [00:00<00:00, 537.49it/s]


TypeError: unhashable type: 'list'

In [43]:
host_activity[-1]

{'PC-5565': ['{Y0A4-H1YL52RQ-3185VBZR}',
  '{U0U0-Z3TJ40RO-4444CPOP}',
  '{V8X6-K8DL10YQ-2855MASK}'],
 'PC-0783': ['{H8X9-F3EE87AR-7906OGRZ}',
  '{H9R9-A9HB24QO-6176ZXLV}',
  '{D6F0-W4VK76MC-9167MLRQ}'],
 'PC-0765': ['{L2L2-S1KZ66VU-4509IIDB}', '{Q8I7-J2KM90AE-1480AEEL}'],
 'PC-2009': ['{K8O6-Q0GC86VF-1536AAUT}', '{S2L3-Y7RR70FH-8820IDCD}'],
 'PC-9155': ['{X8Y2-S6IG70VE-2437JWPD}',
  '{I5Z1-J0AO69VJ-0812NPXT}',
  '{O4X5-N6PC63SN-0756OYNH}'],
 'PC-1262': ['{G6I8-Q2GQ66CS-6507WTCW}', '{X1C4-S6HS97SB-9398VQDV}'],
 'PC-9843': ['{I6W1-W3MG71AW-4776ITNU}',
  '{A1B1-H9DA74ST-5804OUOA}',
  '{S5Z3-F3GW36NT-7369QVFO}'],
 'PC-1275': ['{E5Y8-B0AL13TY-9228MQOQ}', '{N7E0-B0TE51UH-9012ZKUV}'],
 'PC-9002': ['{D9C6-C0PM75RT-4935YGQX}', '{D4P6-T6MW49EF-6599TLDE}'],
 'PC-3195': ['{Y1I9-U1LO50BH-2864MVHU}', '{N9S7-A0DO76QF-3535KEUQ}'],
 'PC-7292': ['{I6V9-S2ED71QA-5686DJSS}', '{E3G9-J7ZH92WE-9748SBZK}'],
 'PC-9290': ['{I4F9-F5MX16IH-4292GSMS}', '{O1N6-T4XD41WN-4420LALN}'],
 'PC-5335': ['{Y9Z7-X7DP20UN-190

In [6]:
# Check 
print(host_activity[0]['PC-9950'])

print(host_activity[1]['PC-9950'])

print(activity_graph[host_activity[1]['PC-9950'][0]])


['{H8X9-F3EE87AR-7906OGRZ}', '{Y3E5-O5JH43YP-2140MCDI}', '{V8T4-X1BO44XW-6300FEJL}', '{H3C3-O6WG03OY-0619DOQP}', '{Z8L6-O4RO26OO-0236TYAG}', '{M4B7-X8KB96EZ-6502LWAW}', '{B9T8-O8GP76AG-5890LFCB}', '{Z8K8-N6FD13EU-1547IJKK}', '{G3Z2-X2HZ41PF-4739KFTU}', '{Q9I1-K3QL90TL-8552HBKA}', '{K1N2-U2SK21AR-7192IIQF}', '{P7V9-A3OA41IG-8730MTTL}', '{M8Z2-K4CM26FD-2054FKGT}', '{S3P9-R7WM46BI-2267MZOH}', '{I4E6-K7HN90FQ-5248JXGM}', '{Q1X2-Z0MX16AD-2287SLBL}', '{Z7K0-E2UH16SM-8696WPKA}', '{V2W1-B2BB60VH-8198OJJM}', '{R8K9-C4IU76HA-7025BZNA}', '{E5L6-C0ZI10KF-8661PXMK}', '{R3O7-H4RE54UM-2774SMJG}', '{B1T5-W0XH95QK-9519TESR}', '{E2B3-P5UC96II-7420FAIF}', '{E3L0-L8XF74DW-6334NDPQ}', '{T7E8-L8NU53MH-1998PEIX}', '{W3O1-V3UH44WW-1478ZGBF}', '{Y1I2-T6EA38GF-0399QWTW}', '{I2I8-E6TK18AL-6659QULD}', '{M7H8-N0QT19ZV-7321AKAC}', '{B7P7-T0AK43MP-0237OGPI}', '{Z4H4-Q9LI46WA-4961DYUA}']
['{E4U3-S2ED81TV-8881XRGN}']
{'{H8X9-F3EE87AR-7906OGRZ}': {0: {'EdgeType': 2, 'weight': 0.03225806451612903}}, '{Z4H4-Q9LI46WA-4961

In [10]:
activity_graph.nodes['{Y3E5-O5JH43YP-2140MCDI}']
for node_id in host_activity[0]['PC-9950']:
    print(activity_graph.nodes[node_id]['H'], activity_graph.nodes[node_id]['A'], activity_graph.nodes[node_id]['obj'])


PC-0783 visit http://nfl.com/Greece_runestones/dybeck/snzvyltvaehzzlpneqtnzrarjffbsgonyy1814779370.jsp
PC-9950 Logon PC-9950
PC-9950 Logon PC-9950
PC-9950 File Open R:\E5S5QBXU.doc
PC-9950 File Write R:\WTC0699\MKE4S35D.pdf
PC-9950 File Write R:\JCE2TLZ7.doc
PC-9950 File Write R:\JCE2TLZ7.doc
PC-9950 File Open R:\WTC0699\VMMYIW17.pdf
PC-9950 File Open R:\E5S5QBXU.doc
PC-9950 File Open R:\WTC0699\81C6ZB46.doc
PC-9950 File Write R:\WTC0699\81C6ZB46.doc
PC-9950 File Open R:\E5S5QBXU.doc
PC-9950 File Write R:\WTC0699\VMMYIW17.pdf
PC-9950 File Open C:\72V18D6\GNE9YOJU.doc
PC-9950 File Write R:\WTC0699\OCYZ2BIQ.doc
PC-9950 File Open R:\E5S5QBXU.doc
PC-9950 File Open C:\HQWSMNKX.doc
PC-9950 File Open R:\WTC0699\HBX5O9TA.doc
PC-9950 File Write R:\SC0B4VLF.jpg
PC-9950 File Open R:\WTC0699\VMMYIW17.pdf
PC-9950 File Open R:\E5S5QBXU.doc
PC-9950 File Open C:\PGRGEP4R.pdf
PC-9950 File Write R:\SC0B4VLF.jpg
PC-9950 File Open R:\E1CXYRD8.zip
PC-9950 File Write R:\WTC0699\81C6ZB46.doc
PC-9950 File Wri

In [57]:
activity_graph.nodes['{E4U3-S2ED81TV-8881XRGN}']

{'type': 'file',
 'sub': 'WTC0699',
 'obj': 'R:\\JCE2TLZ7.doc',
 'A': 'File Write',
 'T': 1262567142.0,
 'H': 'PC-9950'}

In [75]:
print(len(host_activity[0]['PC-9950']))
print(len(host_activity[1]['PC-9950']))
h = []
for key in host_activity[0]:
    h.append(key)
h[:10]

31
1


['PC-3971',
 'PC-4302',
 'PC-0765',
 'PC-2009',
 'PC-2320',
 'PC-9155',
 'PC-9950',
 'PC-1262',
 'PC-9843',
 'PC-1713']

In [79]:
# host = 'PC-9950'
activity_set = set()
for host in h:
    for day_activity in host_activity:
        # print("Day : ", host_activity.index(day_activity))
        if not day_activity or host not in day_activity:
            continue

        for node_id in day_activity[host]:
            # print(host)
            # print(activity_graph.nodes[node_id]['H'], activity_graph.nodes[node_id]['A'])
            activity_set.add(activity_graph.nodes[node_id]['A'])
activity_set

{'File Open', 'File Write', 'Logoff', 'Logon', 'visit'}

In [26]:
activity_graph.nodes[host_activity[0]["PC-9950"][0]]
# host_activity[0]["PC-9950"]

{'type': 'http',
 'sub': 'ZHR0596',
 'obj': 'http://nfl.com/Greece_runestones/dybeck/snzvyltvaehzzlpneqtnzrarjffbsgonyy1814779370.jsp',
 'A': 'visit',
 'T': 1262390519.0,
 'H': 'PC-0783'}

In [33]:
a = [[1,2,3],[4,5,6]]
from itertools import chain
list(chain.from_iterable(a))

[1, 2, 3, 4, 5, 6]

In [35]:
a  = [2,3,45]
a = [[1,2],[3,4]]
for i in a:
    i[1] = 555
a

[[1, 555], [3, 555]]

In [36]:
a = [1,2,3,4,5,6,7]

for num in a:
    if num % 2 == 0:
        a.remove(num)
a

[1, 3, 5, 7]